# Gerekli kütüphaneleri yükleyin

In [1]:
import pandas as pd
from lifetimes import BetaGeoFitter, GammaGammaFitter

# Veri setini oku


In [4]:
file_path = 'flo_data_20k.csv'
df = pd.read_csv(file_path)
df.head()

,master_id,order_channel,last_order_channel,first_order_date,last_order_date,last_order_date_online,last_order_date_offline,order_num_total_ever_online,order_num_total_ever_offline,customer_value_total_ever_offline,customer_value_total_ever_online,interested_in_categories_12
0,cc294636-19f0-11eb-8d74-000d3a38a36f,Android App,Offline,2020-10-30,2021-02-26,2021-02-21,2021-02-26,4.0,1.0,139.99,799.38,[KADIN]
1,f431bd5a-ab7b-11e9-a2fc-000d3a38a36f,Android App,Mobile,2017-02-08,2021-02-16,2021-02-16,2020-01-10,19.0,2.0,159.97,1853.58,"[ERKEK, COCUK, KADIN, AKTIFSPOR]"
2,69b69676-1a40-11ea-941b-000d3a38a36f,Android App,Android App,2019-11-27,2020-11-27,2020-11-27,2019-12-01,3.0,2.0,189.97,395.35,"[ERKEK, KADIN]"
3,1854e56c-491f-11eb-806e-000d3a38a36f,Android App,Android App,2021-01-06,2021-01-17,2021-01-17,2021-01-06,1.0,1.0,39.99,81.98,"[AKTIFCOCUK, COCUK]"
4,d6ea1074-f1f5-11e9-9346-000d3a38a36f,Desktop,Desktop,2019-08-03,2021-03-07,2021-03-07,2019-08-03,1.0,1.0,49.99,159.99,[AKTIFSPOR]


# Tarih değişkenlerini date formatına çevir


In [5]:
date_columns = ["first_order_date", "last_order_date", "last_order_date_online", "last_order_date_offline"]
for col in date_columns:
    df[col] = pd.to_datetime(df[col])

# Aykırı Değerleri Baskılamak için Fonksiyonları Tanımla

In [8]:
def outlier_thresholds(dataframe, variable):
    quartile1 = dataframe[variable].quantile(0.01)
    quartile3 = dataframe[variable].quantile(0.99)
    interquartile_range = quartile3 - quartile1
    up_limit = quartile3 + 1.5 * interquartile_range
    low_limit = quartile1 - 1.5 * interquartile_range
    return low_limit, up_limit

def replace_with_thresholds(dataframe, variable):
    low_limit, up_limit = outlier_thresholds(dataframe, variable)
    dataframe.loc[(dataframe[variable] < low_limit), variable] = low_limit
    dataframe.loc[(dataframe[variable] > up_limit), variable] = up_limit

# Belirtilen Değişkenlerin Aykırı Değerlerini Baskıla


In [9]:
variables = ["order_num_total_ever_online", "order_num_total_ever_offline", 
             "customer_value_total_ever_offline", "customer_value_total_ever_online"]

for variable in variables:
    replace_with_thresholds(df, variable)

# Toplam Alışveriş Sayısı ve Harcaması için Yeni Değişkenler Oluştur


In [10]:
df["total_order"] = df["order_num_total_ever_online"] + df["order_num_total_ever_offline"]
df["total_value"] = df["customer_value_total_ever_online"] + df["customer_value_total_ever_offline"]

# Analiz tarihi

In [11]:
analysis_date = df["last_order_date"].max() + pd.DateOffset(days=2)
print(f'Analiz tarihi: {analysis_date}')

Analiz tarihi: 2021-06-01 00:00:00


# Recency ve Tenure hesaplama (haftalık olarak)

In [18]:
df["recency_cltv_weekly"] = (df["last_order_date"] - df["first_order_date"]).dt.days / 7
df["T_weekly"] = (analysis_date - df["first_order_date"]).dt.days / 7

# Frequency ve Monetary hesaplama

In [19]:
df["frequency"] = df["total_order"]
df["monetary_cltv_avg"] = df["total_value"] / df["total_order"]

# CLTV dataframe'i oluşturma

In [20]:
cltv_df = df.loc[:, ["master_id", "recency_cltv_weekly", "T_weekly", "frequency", "monetary_cltv_avg"]]
cltv_df.columns = ["customer_id", "recency_cltv_weekly", "T_weekly", "frequency", "monetary_cltv_avg"]

# Frequency değerlerinin integer olması gerekiyor


In [21]:
cltv_df["frequency"] = cltv_df["frequency"].astype(int)

# Eksik ve 0 değerleri kontrol et

In [36]:
cltv_df.isnull().sum()
(cltv_df == 0).sum()

cltv_df = cltv_df.dropna()
cltv_df = cltv_df[(cltv_df[['frequency', 'monetary_cltv_avg', 'recency_cltv_weekly']] != 0).all(axis=1)]


# BG/NBD modelini fit et

In [37]:
bgf = BetaGeoFitter(penalizer_coef=0.001)
bgf.fit(cltv_df['frequency'], cltv_df['recency_cltv_weekly'], cltv_df['T_weekly'])

<lifetimes.BetaGeoFitter: fitted with 19920 subjects, a: 0.00, alpha: 76.26, b: 0.00, r: 3.66>

# 3 ay içerisinde beklenen satın almalar


In [23]:
cltv_df['exp_sales_3_month'] = bgf.predict(3*4, cltv_df['frequency'], cltv_df['recency_cltv_weekly'], cltv_df['T_weekly'])

# 6 ay içerisinde beklenen satın almalar


In [24]:
cltv_df['exp_sales_6_month'] = bgf.predict(6*4, cltv_df['frequency'], cltv_df['recency_cltv_weekly'], cltv_df['T_weekly'])

# Gamma-Gamma modelini fit et


In [25]:
ggf = GammaGammaFitter(penalizer_coef=0.01)
ggf.fit(cltv_df['frequency'], cltv_df['monetary_cltv_avg'])

<lifetimes.GammaGammaFitter: fitted with 19945 subjects, p: 4.15, q: 0.47, v: 4.08>

# Tahmin edilen ortalama değeri dataframe'e ekle


In [26]:
cltv_df['exp_average_value'] = ggf.conditional_expected_average_profit(cltv_df['frequency'], cltv_df['monetary_cltv_avg'])

# 6 aylık CLTV hesapla


In [27]:
cltv_df['cltv'] = ggf.customer_lifetime_value(
    bgf, 
    cltv_df['frequency'], 
    cltv_df['recency_cltv_weekly'], 
    cltv_df['T_weekly'], 
    cltv_df['monetary_cltv_avg'], 
    time=6,  # 6 aylık
    freq='W',  # Haftalık
    discount_rate=0.01  # İskonto oranı
)

# Müşterileri CLTV'ye göre segmentlere ayır


In [28]:
cltv_df['cltv_segment'] = pd.qcut(cltv_df['cltv'], 4, labels=["D", "C", "B", "A"])

# DataFrame'i görüntüle


In [29]:
print(cltv_df.head())

                            customer_id  recency_cltv_weekly    T_weekly  \
0  cc294636-19f0-11eb-8d74-000d3a38a36f            17.000000   30.571429   
1  f431bd5a-ab7b-11e9-a2fc-000d3a38a36f           209.857143  224.857143   
2  69b69676-1a40-11ea-941b-000d3a38a36f            52.285714   78.857143   
3  1854e56c-491f-11eb-806e-000d3a38a36f             1.571429   20.857143   
4  d6ea1074-f1f5-11e9-9346-000d3a38a36f            83.142857   95.428571   

   frequency  monetary_cltv_avg  exp_sales_3_month  exp_sales_6_month  \
0          5         187.874000           0.973927           1.947853   
1         21          95.883333           0.983161           1.966323   
2          5         117.064000           0.670586           1.341172   
3          2          60.985000           0.700412           1.400824   
4          2         104.990000           0.396039           0.792077   

   exp_average_value        cltv cltv_segment  
0         193.632662  395.733200            A  
1       

# CLTV değeri en yüksek 20 kişiyi gözlemle


In [30]:
top_20_cltv = cltv_df.sort_values(by='cltv', ascending=False).head(20)
print(top_20_cltv)

                                customer_id  recency_cltv_weekly    T_weekly  \
9055   47a642fe-975b-11eb-8c2a-000d3a38a36f             2.857143    7.857143   
13880  7137a5c0-7aad-11ea-8f20-000d3a38a36f             6.142857   13.142857   
17323  f59053e2-a503-11e9-a2fc-000d3a38a36f            51.714286  101.000000   
12438  625f40a2-5bd2-11ea-98b0-000d3a38a36f            74.285714   74.571429   
8868   9ce6e520-89b0-11ea-a6e7-000d3a38a36f             3.428571   34.428571   
7330   a4d534a2-5b1b-11eb-8dbd-000d3a38a36f            62.714286   67.285714   
6402   851de3b4-8f0c-11eb-8cb8-000d3a38a36f             8.285714    9.428571   
6666   53fe00d4-7b7a-11eb-960b-000d3a38a36f             9.714286   13.000000   
19538  55d54d9e-8ac7-11ea-8ec0-000d3a38a36f            52.571429   58.714286   
14858  031b2954-6d28-11eb-99c4-000d3a38a36f            14.857143   15.571429   
17963  8fd88976-6708-11ea-9d38-000d3a38a36f            50.285714   63.285714   
15516  9083981a-f59e-11e9-841e-000d3a38a

# Aksiyon önerileri


In [31]:
recommendations = {
    "Group A": [
        "Sadakat Programları: En değerli müşterilerinize özel sadakat programları sunun. Özel indirimler, erken erişim fırsatları ve ücretsiz kargo gibi avantajlar sağlayarak sadakati artırın.",
        "Kişiselleştirilmiş Pazarlama: Müşterilerin önceki alışveriş alışkanlıklarına dayanarak kişiselleştirilmiş pazarlama kampanyaları oluşturun. Onlara özel ürün önerileri ve kampanyalar sunarak müşteri memnuniyetini artırın."
    ],
    "Group B": [
        "Çapraz Satış Stratejileri: Bu gruptaki müşterilere ilgili ürünleri sunarak çapraz satış fırsatlarını değerlendirin. Örneğin, bir ayakkabı satın alan müşteriye uyumlu çorap veya bakım ürünleri önerin.",
        "Müşteri Geri Bildirimi: Müşteri geri bildirimlerini toplamak ve değerlendirmek için anketler düzenleyin. Müşteri memnuniyetini artırmak için geri bildirimlere dayalı iyileştirmeler yapın."
    ]
}

# Önerileri görüntüle


In [32]:
print(recommendations)

{'Group A': ['Sadakat Programları: En değerli müşterilerinize özel sadakat programları sunun. Özel indirimler, erken erişim fırsatları ve ücretsiz kargo gibi avantajlar sağlayarak sadakati artırın.', 'Kişiselleştirilmiş Pazarlama: Müşterilerin önceki alışveriş alışkanlıklarına dayanarak kişiselleştirilmiş pazarlama kampanyaları oluşturun. Onlara özel ürün önerileri ve kampanyalar sunarak müşteri memnuniyetini artırın.'], 'Group B': ['Çapraz Satış Stratejileri: Bu gruptaki müşterilere ilgili ürünleri sunarak çapraz satış fırsatlarını değerlendirin. Örneğin, bir ayakkabı satın alan müşteriye uyumlu çorap veya bakım ürünleri önerin.', 'Müşteri Geri Bildirimi: Müşteri geri bildirimlerini toplamak ve değerlendirmek için anketler düzenleyin. Müşteri memnuniyetini artırmak için geri bildirimlere dayalı iyileştirmeler yapın.']}
